<a href="https://colab.research.google.com/github/NadiaCopello/HW6/blob/main/hw6_parte2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

🔴 Nivel avanzado – Comentarios tóxicos


In [1]:
from google.colab import files
uploaded = files.upload()

Saving train.csv.zip to train.csv.zip


In [2]:
import zipfile
import os

# Cambia el nombre si el ZIP se llama diferente
zip_path = "train.csv.zip"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("jigsaw_data")  # Crea una carpeta llamada "jigsaw_data"

In [3]:
import pandas as pd

# Leer datos
df = pd.read_csv("jigsaw_data/train.csv")

# Crear columna binaria en tipo float
df['label'] = df['toxic'].apply(lambda x: 1.0 if x >= 0.5 else 0.0)

# Usamos solo texto y etiqueta
df = df[['comment_text', 'label']]



In [4]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)

# Dividir en entrenamiento y validación
dataset = dataset.train_test_split(test_size=0.1)



In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(example):
    return tokenizer(example["comment_text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["comment_text"])
tokenized_dataset.set_format("torch")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/143613 [00:00<?, ? examples/s]

Map:   0%|          | 0/15958 [00:00<?, ? examples/s]

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)  # regresión binaria


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    logging_dir="./logs",
    logging_steps=10
)



In [8]:
train_data = tokenized_dataset["train"].shuffle(seed=42).select(range(2000))
eval_data = tokenized_dataset["test"].select(range(500))


In [ ]:
# ===============================
# 7. Función para entrenar (CPU / GPU)
# ===============================
from transformers import TrainingArguments, Trainer
import time
import torch

def train_and_time(device):
    model = get_model().to(device)

    training_args = TrainingArguments(
        output_dir=f"./results_{device}",
        num_train_epochs=2,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        evaluation_strategy="epoch",
        save_strategy="no",
        logging_strategy="epoch",
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_tokenized,
        eval_dataset=eval_tokenized
    )

    print(f"🔥 Entrenando en {device.upper()}...")
    start = time.time()
    trainer.train()
    end = time.time()
    print(f"✅ Tiempo total en {device.upper()}: {end - start:.2f} segundos\n")
    return end - start


In [ ]:
#definir metrica de presicion
from sklearn.metrics import accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.sigmoid(torch.tensor(logits)).numpy()
    preds = np.round(probs)
    return {"accuracy": accuracy_score(labels, preds)}


In [ ]:
#Crear callbacks para guardar loss y accuracy
from transformers import TrainerCallback

class SaveMetricsCallback(TrainerCallback):
    def __init__(self):
        self.train_loss = []
        self.eval_accuracy = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            if "loss" in logs:
                self.train_loss.append(logs["loss"])
            if "eval_accuracy" in logs:
                self.eval_accuracy.append(logs["eval_accuracy"])


In [ ]:
cpu_callback = SaveMetricsCallback()
gpu_callback = SaveMetricsCallback()


In [ ]:
#Entrenamiento en CPU y GPU
from transformers import Trainer
from transformers import AutoModelForSequenceClassification

model_cpu = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)

trainer_cpu = Trainer(
    model=model_cpu,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=compute_metrics,
    callbacks=[cpu_callback]
)

trainer_cpu.train()


In [ ]:
#GPU
model_gpu = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)

trainer_gpu = Trainer(
    model=model_gpu,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=compute_metrics,
    callbacks=[gpu_callback]
)

trainer_gpu.train()


In [ ]:
#GRAFICAR
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(cpu_callback.train_loss, label="CPU - Loss", color='blue')
plt.plot(gpu_callback.train_loss, label="GPU - Loss", color='red')
plt.title("Comparación de pérdida durante el entrenamiento")
plt.xlabel("Iteraciones (log steps)")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
#GRAFICAR PRECISION
plt.figure(figsize=(10, 5))
plt.plot(cpu_callback.eval_accuracy, label="CPU - Accuracy", color='blue')
plt.plot(gpu_callback.eval_accuracy, label="GPU - Accuracy", color='red')
plt.title("Comparación de precisión por época")
plt.xlabel("Épocas")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()
